# Data Exploration
In this notebook, we will explore the 3 extra files that we have been provided for the paper `Predicting food insecurity through news streams`. The paper uses IPC classfiications from `fews.net` for ground truth data. 


## Initialisation

In [ ]:
# %pip install pandas folium numpy matplotlib seaborn gdown --break-system-packages --quiet

In [ ]:

import pandas as pd
import numpy as np
import folium
from IPython.display import display, Image
import os
import gdown
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
url = "https://drive.google.com/uc?id=1YoQ1hz9RlaLr2xW3KoKCfJPyyO2PErym"
output = "data.zip"

if not os.path.exists("./data"):
    gdown.download(url, output, quiet=False) 
    zipfile.ZipFile('data.zip', 'r').extractall()
else:
    print("You already have the data downloaded and extracted")

In [ ]:
df_famine_country = pd.read_csv("./data/famine-country-province-district-years-CS.csv")
df_matching_districts = pd.read_csv("./data/matching_districts.csv")
df_nodes = pd.read_csv("./data/fig_1_nodes.csv")

In [ ]:
data = {
    "Seed keyphrase": [
        "famine", "food insecurity", "malnourished", "malnutrition",
        "food crisis", "starvation", "hunger crises", "shortage of food",
        "life-threatening hunger", "lack of food", "scarcity of food",
        "acute hunger", "dearth of food"
    ],
    "Number of articles containing frames": [
        25637, 25404, 12102, 10372, 10154, 8012, 6518, 5482,
        1895, 1266, 1058, 1043, 891
    ]
}

pd.DataFrame(data)


In [ ]:
def pretty_print_list(list_to_print):
    formatted_columns = "\n- " + "\n- ".join(list_to_print)  
    formatted_columns = sorted(list_to_print)
    print("\n- " + "\n- ".join(formatted_columns))

## Famine CSV

In this section, I will try and analyze the data from the `famine-country-province-district-years-CS.csv` file. 

In [ ]:
df_famine_country.head(5)

I think the `Unnamed` column is the index of the data but it is the same as the index of the data frame. So I will drop it.

In [ ]:
df_famine_country.drop(columns=["Unnamed: 0"], inplace=True) # this just seems like a redundant index column

df_columns = pd.DataFrame(df_famine_country.columns.values)

print("\n🗂️ LIST OF ALL COLUMN NAMES IN THE DATASET 🗂️")
print("These are the different attributes available in the dataset for analysis:\n")
df_columns

In [ ]:
print("No. of rows : ", df_famine_country.shape[0])
print("No. of columns : ", df_famine_country.shape[1])

In [ ]:
print("The data ranges from the year ", df_famine_country['year'].min(), " to ", df_famine_country['year'].max())

The paper mentions that they have used data from `from July 2009 to July 2020`. This means that this file does not contain the data for the entire period.

In [ ]:
unique_countries = df_famine_country['country'].unique()
formatted_countries = "\n- " + "\n- ".join(sorted(unique_countries))

print(f"\n🌍 The dataset covers the following {len(unique_countries)} countries:\n")
print(formatted_countries)


The paper says that they analyzed data from 21 countries, but this dataset contains 39 unique countries. Upon manual inspection Congo is mentioned twice so there are only 38 unique countries. This is very close to the mention of 37 countries in the FEWS net dataset as mentioned in paper. I assume this dataset is a subset of the FEWS net dataset.

>  The dataset covers 21 of the 37 countries in the FEWS NET dataset—Afghanistan, Burkina Faso, Chad, Democratic Republic of the Congo, Ethiopia, Guatemala, Haiti, Kenya, Malawi, Mali, Mauritania, Mozambique, Niger, Nigeria, Somalia, South Sudan, Sudan, Uganda, Republic of Yemen, Zambia, and Zimbabwe—over the period from July 2009 to July 2020.

In [ ]:
countries_mentioned_in_paper_set = set(
    ["Afghanistan", "Burkina Faso", "Chad", "Democratic Republic of the Congo", "Ethiopia", 
    "Guatemala", "Haiti", "Kenya", "Malawi", "Mali", "Mauritania", "Mozambique", "Niger", 
    "Nigeria", "Somalia", "South Sudan", "Sudan", "Uganda", "Republic of Yemen", "Zambia", 
    "Zimbabwe"]
)

extra_countries = set(unique_countries) - countries_mentioned_in_paper_set
formatted_difference = "\n- " + "\n- ".join(sorted(list(extra_countries)))
print(f"Countries in dataset that are not mentioned in paper are {formatted_difference}")

missing_countries = countries_mentioned_in_paper_set - set(unique_countries)
formatted_missing = "\n- " + "\n- ".join(sorted(list(missing_countries)))
print(f"\n\nCountries mentioned in paper that are not in dataset are {formatted_missing}")


Okay so upon manual inspection Yemen, Congo are mentioned in the paper but jsut under different names. 

In [ ]:
count_rows_for_country = df_famine_country.groupby(['country']).size().reset_index(name='counts')
count_rows_for_country = count_rows_for_country.sort_values(by='counts', ascending=False)
count_rows_for_country

In [ ]:
count_rows_for_country_per_district = df_famine_country.groupby(['country', 'district']).size().reset_index(name='counts')
count_rows_for_country_per_district

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x='country', data=df_famine_country, order = df_famine_country['country'].value_counts().index)
plt.xticks(rotation=90)
plt.xlabel('Country')
plt.ylabel('Number of rows')
plt.title('Number of rows per country')
plt.show()

This to me seems like it is a dataset of famines in different countries and their district by years and dates. I am still confised by what `CS` column means. It could be IPC classification but that has 5 categories only from 1 to 5 whereas this column has values like 88 and 0.

In [ ]:
df_unique_cs = pd.DataFrame(df_famine_country["CS"].unique(), columns=["Unique CS Values"])
df_value_counts = df_famine_country["CS"].value_counts().reset_index()
df_value_counts.columns = ["CS Value", "Count"]
df_columns = pd.DataFrame(df_famine_country.columns, columns=["Dataset Columns"])

print("\n🌟 UNIQUE VALUES IN THE 'CS' COLUMN 🌟")
print("Below are all unique values found in the 'CS' column, which might represent different classifications or severity levels:\n")
df_unique_cs


In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=df_value_counts["CS Value"], y=df_value_counts["Count"])
plt.title("DISTRIBUTION OF 'CS' VALUES")
plt.xlabel("Unique 'CS' Values")
plt.show()

Maybe it is a misclassification of the data? But upon inspecting more, there are about 4000 rows of data and it is not possible to be a misclassification. So I am still not sure what this column means.

In [ ]:
print("\n📊 FREQUENCY DISTRIBUTION OF COLUMN 📊")
print("This table shows how frequently each unique value appears in the dataset, helping us understand the data distribution:\n")
df_value_counts

### Final Verdict (Subject to change)

The `famine-country-province-district-years-CS.csv` file is used for administrative-level mapping, ensuring consistency across different datasets. It maps districts, provinces, and countries, aligning data from sources like FEWS NET and news reports to the correct geographic regions. The file helps detect missing or misnamed locations.

## Matching districts CSV

In this section, we will analyze the data from the `matching-districts.csv` file.

In [ ]:
df_matching_districts.head(5)

This looks like a relatively small dataset as compared to the last one

In [ ]:
print("No. of rows : ", df_matching_districts.shape[0])
print("No. of columns : ", df_matching_districts.shape[1])

Just by looking at the head of the data this seems to me like it is some sort of correction of either the district names or province names in the dataset depending on the `match` column. Like the missing and district columns have more less the same values for each row so I think this is a correction of the district names. By analyzing the DF more in Data Wrangler, my suspicions have been proved right. (I think)

In [ ]:
display(Image(filename="matching.png"))

### Final Verdict (Subject to change)
The **`matching_districts.csv`** file appears to contain **mapped corrections** or **standardized district names**, likely used to clean or align geographic data in the study.

- The dataset consists of **four columns**:
  1. **`missing`** → District names that were **incorrect, misspelled, or not found** in the main dataset.
  2. **`district`** → The **corrected district name** that aligns with official records or another dataset.
  3. **`province`** → The **province or administrative region** to which the district belongs.
  4. **`match`** → Specifies whether the correction refers to a **district** or potentially another administrative level.

This dataset is likely used for **data cleaning** to ensure that all districts match a **consistent naming standard**.

---

##### **🔍 Column-by-Column Breakdown**
| **Column Name** | **Likely Meaning** |
|---------------|----------------|
| **`missing`** | Contains district names that were **inconsistent, misspelled, or missing** from the main dataset. |
| **`district`** | The **corrected or standardized** district name that replaces the missing one. |
| **`province`** | The **province or region** associated with the corrected district. |
| **`match`** | Indicates whether the correction applies to a **district** or potentially another entity (e.g., province). |

---

This dataset helps by:
✅ **Fixing inconsistencies** in district names across different datasets.  
✅ **Ensuring accurate matching** of districts to food insecurity data.  
✅ **Improving model accuracy**, as mismatched districts could lead to errors in food crisis predictions.

> As can be seen above in the data wrangler Screenshot, the columns do have some weird encoding leading to many escaped characters.  
> I will have to clean this up (later).


## Nodes CSV

In this section, we will analyze the data from the `fig_1_nodes.csv` file. This file contains the 167 extracted text features in the `labels` column along with their frequency in the `size` column.

In [ ]:
df_nodes.head(5)

In [ ]:
df_nodes[df_nodes["label"]=="rise"] 

In the supplemental material, the number of articles for this term are 1,377,032. So maybe the size column is not the number of articles?

In [ ]:
df_nodes.columns.values

In [ ]:
print("No. of rows : ", df_nodes.shape[0])
print("No. of columns : ", df_nodes.shape[1])

In [ ]:
df_nodes.groupby(df_nodes["type"]).size().reset_index(name='counts')

In [ ]:
df_nodes.groupby(df_nodes["cluster"]).size().reset_index(name='counts')

The number of clusters is the same as mentioned in the paper. 
> To assess the content validity of these 167 features, we partition them into 12 semantically distinct clusters. Text features belonging to the same clusters co-occur in the news about twice as frequently as those in different clusters

In [ ]:
df_nodes.sort_values(by='size', ascending=False).head(5) # top 5 nodes by size